<a href="https://colab.research.google.com/github/ThierrryScotto/fer_recognition_using_frequency_neural_network/blob/master/basic_FreNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing libraries

In [1]:
pip install scipy

In [19]:
pip install -q pillow

In [21]:
pip install -q dlib

In [20]:

!pip install -q face_recognition Pillow numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.1/100.1 MB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

from scipy.fftpack import dct

# Importing dataset

In [3]:
# df_ck_plus = pd.read_csv('./Datasets/ckextended.csv')
URL = "https://raw.githubusercontent.com/ThierrryScotto/fer_recognition_using_frequency_neural_network/master/datasets/ckextended.csv"
df_ck_plus = pd.read_csv(URL)
df_ck_plus

,emotion,pixels,Usage
0,6,36 39 35 25 19 11 8 7 3 13 15 9 21 57 75 90 10...,Training
1,6,88 74 19 4 5 5 3 12 8 21 15 21 15 18 24 29 32 ...,Training
2,6,9 2 4 7 1 1 1 0 7 29 49 76 115 141 156 169 177...,Training
3,6,104 106 108 104 95 50 60 61 58 83 126 133 139 ...,Training
4,6,68 72 67 67 6 2 1 1 1 1 1 14 24 24 38 65 79 94...,Training
...,...,...,...
915,5,87 86 88 92 92 127 231 248 251 253 254 254 254...,PrivateTest
916,5,21 24 26 28 27 28 30 8 0 0 0 0 0 0 1 4 37 42 4...,PrivateTest
917,5,76 40 31 38 28 34 38 36 41 36 46 38 44 26 45 5...,PrivateTest
918,5,114 87 16 29 17 25 30 34 37 35 45 93 63 80 73 ...,PrivateTest


## Download of shape_predictor

In [25]:
import os
import bz2

# URL do arquivo shape_predictor_68_face_landmarks.dat.bz2
url = "http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2"

# Caminho para salvar o arquivo compactado
compressed_dat_path = "/content/model/shape_predictor_68_face_landmarks.dat.bz2"

# Caminho para salvar o arquivo descompactado
dat_path = "/content/model/shape_predictor_68_face_landmarks.dat"

os.makedirs('/content/model/', exist_ok=False)

# Baixar o arquivo usando wget
!wget -O {compressed_dat_path} {url}

# Descompactar o arquivo
with bz2.BZ2File(compressed_dat_path) as f_in:
    with open(dat_path, 'wb') as f_out:
        f_out.write(f_in.read())

print(f"Arquivo descompactado salvo em: {dat_path}")

# Verificar se o arquivo foi descompactado corretamente
if os.path.exists(dat_path):
    print("Arquivo descompactado com sucesso!")
else:
    print("Falha ao descompactar o arquivo.")


--2024-07-28 19:41:15--  http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
Resolving dlib.net (dlib.net)... 107.180.26.78
Connecting to dlib.net (dlib.net)|107.180.26.78|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64040097 (61M)
Saving to: ‘/content/model/shape_predictor_68_face_landmarks.dat.bz2’

/content/model/shap 100%[===================>]  61.07M  44.5MB/s    in 1.4s    

2024-07-28 19:41:17 (44.5 MB/s) - ‘/content/model/shape_predictor_68_face_landmarks.dat.bz2’ saved [64040097/64040097]

Arquivo descompactado salvo em: /content/model/shape_predictor_68_face_landmarks.dat
Arquivo descompactado com sucesso!


## Converting 1D vectors to images

In [17]:
HEIGHT = 48
WIDTH = 48
OUTPUT_FOLDER = '/content/images/input_images'
os.makedirs(OUTPUT_FOLDER, exist_ok=False)
OUTPUT_PATH = os.path.join(OUTPUT_FOLDER, 'reconstructed_image')

In [6]:
df_ck_plus.shape

(920, 3)

In [18]:
for index, row in df_ck_plus.iterrows():
  one_dimensional_str = row['pixels'].split()
  one_dimensional_vec = np.array(list(map(int, one_dimensional_str)))

  if one_dimensional_vec.size == HEIGHT * WIDTH:
    image_2d = one_dimensional_vec.reshape((HEIGHT, WIDTH))
    plt.imsave(f'{OUTPUT_PATH}_{index}.png', image_2d, cmap='gray')
  else:
    print()
    raise ValueError(f"The size of the vector {one_dimensional_vec.size} does not correspond to the product of the dimensions ({HEIGHT}, {WIDTH})")

#Image pre-processing

In [26]:
import dlib
from PIL import Image
import numpy as np
import os

def adjust_and_crop_face(image_path, output_folder):
    # Carregar a imagem
    image = dlib.load_rgb_image(image_path)

    # Inicializar o detector de rostos e o preditor de pontos-chave
    detector = dlib.get_frontal_face_detector()

    # Caminho completo para o arquivo shape_predictor_68_face_landmarks.dat
    dat_file_path = '/content/model/shape_predictor_68_face_landmarks.dat'
    # Carregar o preditor de forma
    predictor = dlib.shape_predictor(dat_file_path)

    # Detectar os rostos na imagem
    faces = detector(image, 1)

    # Criar a pasta de saída, se não existir
    os.makedirs(output_folder, exist_ok=True)

    # Processar cada rosto detectado
    for i, face in enumerate(faces):
        # Obter os pontos chave do rosto
        landmarks = predictor(image, face)

        # Converter os pontos chave para um formato utilizável
        face_landmarks = {
            "left_eye": [(landmarks.part(j).x, landmarks.part(j).y) for j in range(36, 42)],
            "right_eye": [(landmarks.part(j).x, landmarks.part(j).y) for j in range(42, 48)],
            "left_eyebrow": [(landmarks.part(j).x, landmarks.part(j).y) for j in range(17, 22)],
            "right_eyebrow": [(landmarks.part(j).x, landmarks.part(j).y) for j in range(22, 27)],
            "chin": [(landmarks.part(j).x, landmarks.part(j).y) for j in range(0, 17)],
        }

        # Obter os pontos dos olhos, sobrancelhas e queixo
        left_eye = np.array(face_landmarks['left_eye'])
        right_eye = np.array(face_landmarks['right_eye'])
        left_eyebrow = np.array(face_landmarks['left_eyebrow'])
        right_eyebrow = np.array(face_landmarks['right_eyebrow'])
        chin = np.array(face_landmarks['chin'])

        # Calcular o ponto médio entre os olhos
        left_eye_center = left_eye.mean(axis=0)
        right_eye_center = right_eye.mean(axis=0)
        eye_center = ((left_eye_center + right_eye_center) / 2).astype(int)

        # Calcular o ângulo para horizontalizar os olhos
        dy = right_eye_center[1] - left_eye_center[1]
        dx = right_eye_center[0] - left_eye_center[0]
        angle = np.degrees(np.arctan2(dy, dx))

        # Rotacionar a imagem para horizontalizar os olhos
        pil_image = Image.fromarray(image)
        pil_image = pil_image.rotate(angle, center=tuple(eye_center), expand=True)

        # Recalcular os pontos chave após rotação
        rotated_image = np.array(pil_image)
        rotated_faces = detector(rotated_image, 1)
        rotated_landmarks = predictor(rotated_image, rotated_faces[0])

        # Obter os novos pontos chave para sobrancelhas e queixo
        left_eyebrow = np.array([(rotated_landmarks.part(j).x, rotated_landmarks.part(j).y) for j in range(17, 22)])
        right_eyebrow = np.array([(rotated_landmarks.part(j).x, rotated_landmarks.part(j).y) for j in range(22, 27)])
        chin = np.array([(rotated_landmarks.part(j).x, rotated_landmarks.part(j).y) for j in range(0, 17)])

        # Calcular o topo das sobrancelhas e o fundo do queixo
        top_of_eyebrows = min(left_eyebrow[:,1].min(), right_eyebrow[:,1].min())
        bottom_of_chin = chin[:,1].max()

        # Calcular as coordenadas do recorte
        top = top_of_eyebrows
        bottom = bottom_of_chin
        left = min(chin[:,0].min(), left_eyebrow[:,0].min(), right_eyebrow[:,0].min())
        right = max(chin[:,0].max(), left_eyebrow[:,0].max(), right_eyebrow[:,0].max())

        # Recortar a imagem
        cropped_face = pil_image.crop((left, top, right, bottom))

        # Salvar a imagem recortada com o nome original acrescido de "face"
        base_filename, ext = os.path.splitext(os.path.basename(image_path))
        face_image_path = os.path.join(output_folder, f'{base_filename}_face{ext}')
        cropped_face.save(face_image_path)
        print(f'Rosto {i+1} salvo em {face_image_path}')

# Caminho para a pasta de entrada
input_folder = '/content/images/input_images/'

# Pasta para salvar os rostos recortados
output_folder = '/content/images/preprocessed_images'

# Processar cada imagem na pasta de entrada
for filename in os.listdir(input_folder):
    if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
        image_path = os.path.join(input_folder, filename)
        adjust_and_crop_face(image_path, output_folder)


Rosto 1 salvo em /content/images/preprocessed_images/reconstructed_image_390_face.png
Rosto 1 salvo em /content/images/preprocessed_images/reconstructed_image_237_face.png
Rosto 1 salvo em /content/images/preprocessed_images/reconstructed_image_439_face.png
Rosto 1 salvo em /content/images/preprocessed_images/reconstructed_image_268_face.png
Rosto 1 salvo em /content/images/preprocessed_images/reconstructed_image_540_face.png
Rosto 1 salvo em /content/images/preprocessed_images/reconstructed_image_593_face.png
Rosto 1 salvo em /content/images/preprocessed_images/reconstructed_image_552_face.png
Rosto 1 salvo em /content/images/preprocessed_images/reconstructed_image_639_face.png
Rosto 1 salvo em /content/images/preprocessed_images/reconstructed_image_122_face.png
Rosto 1 salvo em /content/images/preprocessed_images/reconstructed_image_207_face.png
Rosto 1 salvo em /content/images/preprocessed_images/reconstructed_image_269_face.png
Rosto 1 salvo em /content/images/preprocessed_images/r

In [29]:
from google.colab import drive

# Montar o Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [38]:
import shutil
import os

def copy_folder(src, dst):
    # Criar a pasta de destino se não existir
    os.makedirs(dst, exist_ok=True)

    # Iterar sobre todos os arquivos e subpastas na pasta de origem
    for item in os.listdir(src):
        src_path = os.path.join(src, item)
        dst_path = os.path.join(dst, item)

        # Se for um diretório, fazer a cópia recursiva
        if os.path.isdir(src_path):
            copy_folder(src_path, dst_path)
        else:
            shutil.copy2(src_path, dst_path)

# Caminho da pasta de origem no Colab
# src_folder = '/content/images/preprocessed_images'
src_folder = '/content/images/input_images'

# Caminho da pasta de destino no Google Drive
# dst_folder = '/content/drive/MyDrive/trabalho_processamento_imagem/preprocessed_images'
dst_folder = '/content/drive/MyDrive/trabalho_processamento_imagem/input_images'

# Copiar o conteúdo da pasta
copy_folder(src_folder, dst_folder)

print(f'Conteúdo copiado de {src_folder} para {dst_folder}')


Conteúdo copiado de /content/images/input_images para /content/drive/MyDrive/trabalho_processamento_imagem/input_images


# DCT and CIE Layer

Getting one image of example

In [60]:
def load_image_as_array(image_path):
    image = Image.open(image_path).convert('L')  # Load grayscale image
    return np.array(image)

In [61]:
# Caminhos das imagens
image_paths = [
    '/content/images/preprocessed_images/reconstructed_image_0_face.png',
    '/content/images/preprocessed_images/reconstructed_image_1_face.png',
    '/content/images/preprocessed_images/reconstructed_image_2_face.png',
    '/content/images/preprocessed_images/reconstructed_image_3_face.png',
    '/content/images/preprocessed_images/reconstructed_image_4_face.png',
    '/content/images/preprocessed_images/reconstructed_image_5_face.png',
    '/content/images/preprocessed_images/reconstructed_image_6_face.png'
]

# Lista para armazenar os arrays das imagens
image_arrays = []

# Carregar cada imagem e convertê-la em um array NumPy
for image_path in image_paths:
    if os.path.exists(image_path):
        image_array = load_image_as_array(image_path)
        image_arrays.append(image_array)
    else:
        print(f"Imagem não encontrada: {image_path}")

print(len(image_arrays))

7


**using the gamma parameter to reduce the image size**

In [62]:
γ = 0.25

def resize_dct(dct_image, gamma):
    h, w = dct_image.shape
    new_h, new_w = int(h * gamma), int(w * gamma)
    return dct_image[:new_h, :new_w]

verificando a escala das imagens

In [65]:
# for i, image_array in enumerate(image_arrays):
#     print(f"Forma da imagem {i}: {image_array.shape}")

# # Exibir a quantidade de imagens carregadas
# print(f"Total de imagens carregadas: {len(image_arrays)}")

In [64]:
frequency_input = []

for index in range(len(image_arrays)):
    transformed_image = dct(dct(image_arrays[index].T, norm='ortho').T, norm='ortho')
    frequency_input.append(resize_dct(transformed_image, γ))
    # frequency_input.append(dct(dct(image_arrays[index].T, norm='ortho').T, norm='ortho'))

print(frequency_input[0])

[[ 3.72383526e+03 -3.44326340e+02 -2.30385754e+01  7.59917893e+00
  -8.34164493e+01  1.52066168e+01 -1.86708417e+02  1.11492957e+02
   2.91823760e+00  8.82425553e+01]
 [-2.88763986e+02 -2.82724038e+02 -9.44295291e+01 -2.82699360e+02
   1.58901081e+02 -2.21847285e+02 -2.89764579e+02 -1.85962737e+02
  -1.27069177e+02 -1.05911573e+02]
 [-1.72231264e+02  2.24352394e+02 -7.22937204e+01  1.72610862e+02
   3.09199663e+02  2.14247838e+02  8.74025587e+00  1.34367857e+02
  -3.36740512e+01  6.89095936e+01]
 [-2.58464279e+02 -5.63159564e+01 -4.42567003e+01 -1.19371493e+02
  -2.07416436e+02 -5.45476975e+01 -5.88160763e+01  9.61698285e+00
  -6.85065647e+01  4.84519084e+01]
 [ 4.31677248e+00  4.24496551e+01 -9.70223345e+01  3.30290654e+01
   5.93772803e+01  3.01244436e+01  2.75365189e+01 -3.62521198e+01
  -3.07172160e+00 -5.75987171e+01]
 [ 3.88184883e+01  7.61617415e+00 -1.27861380e+02  2.82805798e+01
  -1.88841796e+02  6.86708969e+01 -2.95374381e+01  4.91549538e+01
   4.41905735e+01  7.66645209e+01

# Basic-FreNet

**Paremeters**

In [50]:
n = 5
k = (40, 1, 1, 1, 1)
conv = 5
c = 6